# XML Data Cleaning

[Rekordbox](https://rekordbox.com/en/) is the primary DJing software that I use to manage my song library. It's made by Pioneer, the leading company in the DJing business. Rekordbox can also be used for actual DJing but I never do because I either use my own standalone unit called the XDJ-RX3 or I use the CDJs provided by the venue. I used to use Serato but migrated to Rekordbox because it's more compatible with the gear I use.

I use rekordbox to clean up the metadata of my song files, organize them based my own DJ taxonomy, and compile a setlist of songs for gigs.

![sd]("../imgs/rekordbox_full.png")

**XML Cleaning Export**

The great thing about rekordbox is that I can export a XML file containing a comprehensive collection of all the song data from rekordbox. This not just includes metadata about songs (title, artist, album, etc...) but also the playlist data as well. That way if I want, I can use my own manually-annotated labels (genre, energy, region) in my analysis. If I ever to train my own model, I can readily-available labels.

In this notebook I demonstrate how I use the rekordbox xml data to fix dirty or missing metadata in my database. I also do some lighting data munging/handling with the playlist data as well.

In [11]:
#Imports
import xml.etree.ElementTree as ET
import json
import xmltodict
from urllib import parse
import unicodedata
import pathlib
from tqdm import tqdm
import pandas as pd
import numpy as np
import sqlite3

In [12]:
#Imports from project package called project_tools
from project_tools.utils import adapt_array, convert_array, tag_cleaner, json_opener, table_loader

In [13]:
#Register these functions with sqlite3 so that we I can work with 
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

In [14]:
#Connect to db
conn = sqlite3.connect("../jaage.db", detect_types= sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

Import the `tags` table joined with `files`, which is needed for connecting to the rekordbox data because it doesn't have the sid column.

In [63]:
query = """
SELECT  t.sid, t.artist, t.length, t.title, t.bpm, t.initialkey, f.file_path
FROM tags as t
INNER JOIN files as f ON f.sid = t.sid"""

tags = pd.read_sql_query(query, con = conn)

In [64]:
tags.head()

,sid,artist,length,title,bpm,initialkey,file_path
0,b806881a54bdbf9dd93a290716adf191,,287.393372,House Of Love_PN,119.0,6A,/Volumes/LaCie/Collection/04 House Of Love_PN.wav
1,46e54d2ab920a088b77382e04877141b,A.T.S,311.251892,Baa Daa Laa (Alex Virgo's Rough n Ready edit),128.0,1A,/Volumes/LaCie/Collection/Alex Virgo - Rough N...
2,a204ddef5763df6d8f7677701fe9d96f,FROM BEYOND,415.114746,Protostar,117.0,5A,/Volumes/LaCie/Collection/01 Protostar_PN.wav
3,960097894e83c5810a9c649f17a4e551,Cristal,321.108765,Drink My Soul (Running Hot Edit)_PN,120.0,5A,/Volumes/LaCie/Collection/Cristal - Drink My S...
4,a3c1f277aa0110ffc418bf5fa3aa16aa,Maya,378.276276,Lait De Coco ( Les Yeux Orange Edit)_PN,109.0,6A,/Volumes/LaCie/Collection/Maya - Lait De Coco ...


Load in the rekordbox xml data and parse it with the `xmltodict` package.

In [18]:
with open("rekordbox_data.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

### Cleaning

The `data_dict` object is a nested dictionary with a number of layers to it.

So what I usually like to do is start from the outer later and develop an understanding by parsing it step by step

Outer dictionary has one key in it, which points to another dictionary.

In [19]:
print(data_dict.keys(), type(data_dict["DJ_PLAYLISTS"]))

dict_keys(['DJ_PLAYLISTS']) <class 'dict'>


That dictinoary has four keys

In [11]:
data_dict["DJ_PLAYLISTS"].keys()

dict_keys(['@Version', 'PRODUCT', 'COLLECTION', 'PLAYLISTS'])

First up let's take a look at "COLLECTION" which holds the metadata of all my songs

In [21]:
collection = data_dict["DJ_PLAYLISTS"]["COLLECTION"]
type(collection)

dict

Show the keys in the `collection` dictionary 

In [22]:
collection.keys()

dict_keys(['@Entries', 'TRACK'])

"@Entries" shows the number of tracks

In [23]:
collection["@Entries"]

'3961'

The "TRACK" key points to a list of song data

In [25]:
type(collection["TRACK"])

list

Here's a single example of the "TRACK" data.

In [26]:
collection["TRACK"][0]

{'@TrackID': '122882692',
 '@Name': "Ella Elle L'a (Folamour Edit)",
 '@Artist': 'France Gall',
 '@Composer': '',
 '@Album': '',
 '@Grouping': '',
 '@Genre': '',
 '@Kind': 'WAV File',
 '@Size': '98848262',
 '@TotalTime': '373',
 '@DiscNumber': '0',
 '@TrackNumber': '0',
 '@Year': '0',
 '@AverageBpm': '107.00',
 '@DateAdded': '2022-01-15',
 '@BitRate': '2116',
 '@SampleRate': '44100',
 '@Comments': '',
 '@PlayCount': '3',
 '@Rating': '0',
 '@Location': 'file://localhost/Users/georgemcintire/Music/iTunes/iTunes%20Media/Music/Unknown%20Artist/Unknown%20Album/France%20Gall%20-%20Ella%20Elle%20L%27a%20(Folamour%20Edit).wav',
 '@Remixer': 'Folamour',
 '@Tonality': '9A',
 '@Label': '',
 '@Mix': '',
 'TEMPO': {'@Inizio': '0.000',
  '@Bpm': '107.00',
  '@Metro': '4/4',
  '@Battito': '3'}}

Whenever I work with this kind of json data, I always load it into a pandas dataframe using pandas' `json_normalize` method.

In [29]:
rbox = pd.json_normalize(collection["TRACK"])
rbox.head()

,@TrackID,@Name,@Artist,@Composer,@Album,@Grouping,@Genre,@Kind,@Size,@TotalTime,...,TEMPO.@Battito,TEMPO,POSITION_MARK.@Name,POSITION_MARK.@Type,POSITION_MARK.@Start,POSITION_MARK.@Num,POSITION_MARK.@Red,POSITION_MARK.@Green,POSITION_MARK.@Blue,POSITION_MARK
0,122882692,Ella Elle L'a (Folamour Edit),France Gall,,,,,WAV File,98848262,373,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,135428827,Alors On Danse_(Extended Mix)-instrumental,Stromae,,,,,WAV File,45585630,258,...,NaN,"[{'@Inizio': '0.377', '@Bpm': '119.98', '@Metr...",,0,0.377,-1,NaN,NaN,NaN,NaN
2,90601945,Alors On Danse_(Extended Mix)-harmonic,Stromae,,,,,WAV File,45585626,258,...,NaN,"[{'@Inizio': '0.125', '@Bpm': '119.97', '@Metr...",,0,0.625,-1,NaN,NaN,NaN,NaN
3,97379141,Alors On Danse_(Extended Mix)-drums-vocals,Stromae,,,,,WAV File,45585630,258,...,NaN,"[{'@Inizio': '0.379', '@Bpm': '119.97', '@Metr...",,0,0.379,-1,NaN,NaN,NaN,NaN
4,196901058,Alors On Danse_(Extended Mix)-acappella,Stromae,,,,,WAV File,45585624,258,...,NaN,"[{'@Inizio': '0.124', '@Bpm': '119.99', '@Metr...",,0,0.624,-1,NaN,NaN,NaN,NaN


In [30]:
rbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3961 entries, 0 to 3960
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   @TrackID              3961 non-null   object
 1   @Name                 3961 non-null   object
 2   @Artist               3961 non-null   object
 3   @Composer             3961 non-null   object
 4   @Album                3961 non-null   object
 5   @Grouping             3961 non-null   object
 6   @Genre                3961 non-null   object
 7   @Kind                 3961 non-null   object
 8   @Size                 3961 non-null   object
 9   @TotalTime            3961 non-null   object
 10  @DiscNumber           3961 non-null   object
 11  @TrackNumber          3961 non-null   object
 12  @Year                 3961 non-null   object
 13  @AverageBpm           3961 non-null   object
 14  @DateAdded            3961 non-null   object
 15  @BitRate              3961 non-null   

Get rid of the @ character from the column names.

In [31]:
rbox.columns = rbox.columns.str.strip("@")

Filter out songs not located on my harddrive (LaCie) and select a list of columns 

In [33]:
rbox = rbox[rbox.Location.str.contains("LaCie")]
rekord_cols = ['TrackID', 'Name', 'Artist', "Genre", "Size", "TotalTime", "AverageBpm", "PlayCount", 
              "Location", "TEMPO", "Tonality"]

rbox = rbox[rekord_cols]
rbox.shape

(3939, 11)

Since I can't use the song ids from my database for the joins, I'm going to have to connect `rbox` with `tags` using the file paths. However that presents its own challenges.


First thing I need to deal with is the rekordbox file paths which all begin with "file://localhost/." I don't know it's doing, so if you do, please let me know.

In [36]:
rbox.Location.tolist()[:5]

['file://localhost/Volumes/LaCie/Collection/IORGA%20-%20Romanian%20Edits,%20Vol.%202%20-%2004%20Romanticii%20-%20Soare%20Si%20Foc%20(Iorga%20Edit)_PN.wav',
 'file://localhost/Volumes/LaCie/Collection/Dino%20Soccio%20-%20Continental%20Samba%20DS%20Rework_PN.wav',
 'file://localhost/Volumes/LaCie/Collection/Andrzej%20Korzy%c5%84ski%20-%20Andrzej%20Korzy%c5%84ski%20-%20Temat%20Monika%20(Pejza%c5%bc%20Edit)_PN.wav',
 'file://localhost/Volumes/LaCie/Collection/RAF%20-%20Self%20Control%20(Ziggy%20Phunk%20Spirit%20Of%201984%20Edit)_PN.wav',
 'file://localhost/Volumes/LaCie/Collection/Chemise%20-%20She%20Can%27t%20Love%20You%20(Purple%20Disco%20Machine%20Edit)_PN.wav']

Create a new column called `Location_fixed` by left stripping "file://localhost." I also use `parse.unquote` to deal with the %20 spacing issue as well since those characters don't appear in the `tags` dataframe.

In [37]:
rbox["Location_fixed"] = "/"+rbox.Location.str.lstrip("file://localhost").apply(parse.unquote)

Applying these fixes got me about 80% of the way there in terms cleanly joining data. However, the biggest hurdle to 100% join were the accent characters in the song paths — definitely an issue when working a very global collection of songs.

To address this issue I normalize the columns containing the file paths using the NFKC unicode normalization algorithm.

In [65]:
rbox["Location_unicode"] = rbox.Location_fixed.apply(lambda x: unicodedata.normalize("NFKC", x))
tags["file_path_unicode"] = tags.file_path.apply(lambda x: unicodedata.normalize("NFKC", x))

Now let's see how clean of a join we got here.

For reference, the number of rows in `tags` and `rbox`

In [66]:
tags.shape[0], rbox.shape[0]

(3899, 3939)

Inner join with `tags` and `rbox`

In [68]:
tags_rbox_ij = pd.merge(tags, rbox, how = "inner", left_on="file_path_unicode", right_on="Location_unicode")
tags_rbox_ij.shape

(3899, 21)

Clean join! All of the `tags` data is represented in `tags_rbox_ij`.

In [18]:
joined_outer = pd.merge(tags, df, how = "outer", left_on="file_path_unicode", right_on="Location_unicode")
joined_inner.shape

(3847, 21)

#### Replace missing data

For each column I want to fix in the `tags` table I need to find the rows with missing or incorrect data and replace them with the correct data from their counterpart column in the `rbox` dataframe.


Let's demonstrate this first with bpm.

In [72]:
print(f"There are {tags.bpm.isnull().sum()} null values under bpm column in the tags table")

There are 88 null values under bpm column in the tags table


Select the rows in the `tags_rbox_ij` that are null under the `bpm` column. I set the index to the song id column for reasons I show demonstate after this part.

In [77]:
bpm_null = joined_inner[(joined_inner.bpm.isnull()) & (joined_inner.AverageBpm.notnull()) ].set_index("sid")
bpm_null.head()

,artist,length,title,bpm,initialkey,file_path,file_path_unicode,TrackID,Name,Artist,Genre,Size,TotalTime,AverageBpm,PlayCount,Location,TEMPO,Tonality,Location_fixed,Location_unicode
sid,,,,,,,,,,,,,,,,,,,,
9a10e738a0ce3b630b277647ccb36fa0,Alex Virgo/Jack Priest,445.669128,Here We Go Again,NaN,None,/Volumes/LaCie/Collection/01-Alex_Virgo_Jack_P...,/Volumes/LaCie/Collection/01-Alex_Virgo_Jack_P...,63491102,Here We Go Again,Alex Virgo/Jack Priest,Disco/Nu Disco/Re-Edits,117925428,445,122.00,0,file://localhost/Volumes/LaCie/Collection/01-A...,NaN,8A,/Volumes/LaCie/Collection/01-Alex_Virgo_Jack_P...,/Volumes/LaCie/Collection/01-Alex_Virgo_Jack_P...
df2754c814418e80f40cfdead0ad3e17,C. Da Afro,332.638184,Collectiva's Jam (original mix),NaN,None,/Volumes/LaCie/Collection/01-C._Da_Afro_-_Coll...,/Volumes/LaCie/Collection/01-C._Da_Afro_-_Coll...,56825554,Collectiva's Jam (original mix),C. Da Afro,Disco/Nu Disco/Re-Edits,88017456,332,114.00,0,file://localhost/Volumes/LaCie/Collection/01-C...,NaN,7A,/Volumes/LaCie/Collection/01-C._Da_Afro_-_Coll...,/Volumes/LaCie/Collection/01-C._Da_Afro_-_Coll...
c98685a900ae4b5f9f1880b19688df53,Dam Swindle,476.049988,The Break Up,NaN,None,/Volumes/LaCie/Collection/01-Dam_Swindle_-_The...,/Volumes/LaCie/Collection/01-Dam_Swindle_-_The...,185624456,The Break Up,Dam Swindle,Deep house,125964170,476,120.00,0,file://localhost/Volumes/LaCie/Collection/01-D...,"[{'@Inizio': '0.096', '@Bpm': '120.00', '@Metr...",3A,/Volumes/LaCie/Collection/01-Dam_Swindle_-_The...,/Volumes/LaCie/Collection/01-Dam_Swindle_-_The...
df67ab199063cc81129ff3ad25164301,Dele Sosimi,273.574615,You No Fit Touch Am (Medlar remix radio),NaN,None,/Volumes/LaCie/Collection/01-Dele_Sosimi_-_You...,/Volumes/LaCie/Collection/01-Dele_Sosimi_-_You...,182243122,You No Fit Touch Am (Medlar remix radio),Dele Sosimi,Disco/Nu Disco/Re-Edits,72389300,273,118.00,0,file://localhost/Volumes/LaCie/Collection/01-D...,NaN,7A,/Volumes/LaCie/Collection/01-Dele_Sosimi_-_You...,/Volumes/LaCie/Collection/01-Dele_Sosimi_-_You...
3c0040df6f4c92d73fa0219670f24a0a,Disco Incorporated,308.522461,Giorgio (Disco Progressive mix),NaN,None,/Volumes/LaCie/Collection/01-Disco_Incorporate...,/Volumes/LaCie/Collection/01-Disco_Incorporate...,61427023,Giorgio (Disco Progressive mix),Disco Incorporated,Disco/Nu Disco/Re-Edits,81636444,308,122.00,0,file://localhost/Volumes/LaCie/Collection/01-D...,NaN,4A,/Volumes/LaCie/Collection/01-Disco_Incorporate...,/Volumes/LaCie/Collection/01-Disco_Incorporate...


Create a dictionary where keys are the song ids and the values are the replace bpms which are pulled from rekordBox's `AverageBpm` column

In [78]:
new_bpms = bpm_null.AverageBpm.to_dict()

I use this `updator` function to take in a dictionary of ids and new values along with a table name to update my database with the new data.

In [80]:
def updator(new, tbl):
    update_query = '''UPDATE tags SET {}=? WHERE sid = ? '''.format(tbl)
    
    for ID, val in new.items():
        cur.execute(update_query, (val, ID))
        conn.commit()

Use `updator` to fix the bpms in the tags table.

In [81]:
updator(new_bpms, "bpm")

Check to see if this worked by counting number of nulls in `tags`

In [82]:
pd.read_sql_query("SELECT COUNT(*) as n_nulls FROM tags WHERE bpm is null", con = conn)

,n_nulls
0,0


Yay! It works!!

Time to repeat this for title, artist, and key.

Grab the null rows and create replacement dictionaries.

In [87]:
key_null = tags_rbox_ij[tags_rbox_ij.initialkey.isnull()].set_index("sid")
title_null = tags_rbox_ij[tags_rbox_ij.title.isnull()].set_index("sid")
artist_null = tags_rbox_ij[tags_rbox_ij.artist.isnull()].set_index("sid")

new_keys = key_null.Tonality.to_dict()
new_titles = title_null.Name.to_dict()
new_artists = artist_null.Artist.to_dict()


Call `updator` for keys, titles, and artists

In [88]:
updator(new_titles, "title")
updator(new_artists, "artist")
updator(new_keys, "initialkey")

Check the data

In [89]:
query = """
SELECT COUNT(*) as n_nulls FROM tags WHERE title is null OR artist is null or initialkey is null
"""
pd.read_sql_query(query, con = conn)

,n_nulls
0,0


No nulls again!

I can't emphasize how much a godsend this functionality is for me. I thought I was going to have to retag all my database data the same way I manually tagged it on rekordbox. 

There's a data scientist loves more than clean data amirite?!

### Playlist Data

Now let's dive into the playlist data and understand that information is organized

Grab the playlist data from the rekordbox dictionary data.

In [91]:
playlists = data_dict["DJ_PLAYLISTS"]["PLAYLISTS"]
playlists.keys()

dict_keys(['NODE'])

Use `json_normalize` to parse the playlist data from `playlists["NODE"]["NODE"]`

In [95]:
pl = pd.json_normalize(playlists["NODE"]["NODE"])
pl.columns = pl.columns.str.replace("@", "").str.replace(".", "_")

/opt/miniconda3/envs/music/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [96]:
pl.head()

,Name,Type,KeyType,Entries,TRACK_Key,TRACK,Count,NODE
0,Gigs,1,0,0,NaN,NaN,NaN,NaN
1,Stems and Tools,1,0,1,209794866,NaN,NaN,NaN
2,No artists,1,0,62,NaN,"[{'@Key': '268245690'}, {'@Key': '268245719'},...",NaN,NaN
3,Slate backup,1,0,2,NaN,"[{'@Key': '268247559'}, {'@Key': '268247924'}]",NaN,NaN
4,Give Another Look,1,0,164,NaN,"[{'@Key': '268245456'}, {'@Key': '268245440'},...",NaN,NaN


Some playlists are actually playlist directories. For example "Genres" in my rekordbox app stores playlists, each other referring a different genre.

Playlist directories such as "Genres" don't have null values under the `NODE` column and instead hold nested data for their sub-playlists.

In [100]:
pl[pl.Name == "Genres"]["NODE"]

5    [{'@Name': 'Reggae', '@Type': '1', '@KeyType':...
Name: NODE, dtype: object

For the first go, let's focus on genre data and create a clean dataframe of genre data

Grab the data located at intersection of "GENRES" and "NODE" and convert it to a list.

In [106]:
genre_list = pl[pl.Name == "Genres"]["NODE"].tolist()[0]
genre_list[:2]

[{'@Name': 'Reggae',
  '@Type': '1',
  '@KeyType': '0',
  '@Entries': '3',
  'TRACK': [{'@Key': '268245278'},
   {'@Key': '268246423'},
   {'@Key': '174798994'}]},
 {'@Name': 'Jazz',
  '@Type': '1',
  '@KeyType': '0',
  '@Entries': '9',
  'TRACK': [{'@Key': '268245352'},
   {'@Key': '268245868'},
   {'@Key': '268246121'},
   {'@Key': '268247765'},
   {'@Key': '268246884'},
   {'@Key': '268245343'},
   {'@Key': '268246441'},
   {'@Key': '268246463'},
   {'@Key': '268246851'}]}]

Pass this list of dictionaries into `json_normalize` and remove @ from column names

In [124]:
genres = pd.json_normalize(genre_list).rename(columns= lambda x:x.strip("@")).rename(columns = {"Name":"GenreName"})
genres

,GenreName,Type,KeyType,Entries,TRACK
0,Reggae,1,0,3,"[{'@Key': '268245278'}, {'@Key': '268246423'},..."
1,Jazz,1,0,9,"[{'@Key': '268245352'}, {'@Key': '268245868'},..."
2,Disco,1,0,1001,"[{'@Key': '268246498'}, {'@Key': '163757037'},..."
3,Funk,1,0,214,"[{'@Key': '268246483'}, {'@Key': '268246479'},..."
4,House,1,0,502,"[{'@Key': '268246499'}, {'@Key': '268246481'},..."
5,80s,1,0,107,"[{'@Key': '268246479'}, {'@Key': '205133736'},..."
6,Electro,1,0,305,"[{'@Key': '268246506'}, {'@Key': '268246486'},..."
7,Pop,1,0,109,"[{'@Key': '268246508'}, {'@Key': '268246506'},..."
8,Rock,1,0,35,"[{'@Key': '268247221'}, {'@Key': '268245892'},..."
9,Boogie,1,0,615,"[{'@Key': '163757037'}, {'@Key': '268246486'},..."


Now we have a dataframe that has the names of the playlist, number of songs under each one, and a list rekordbox's track ids of the songs in each genre.

In [121]:
genres = pd.json_normalize(pl[pl.Name == "Genres"]["NODE"].tolist()[0]).rename(columns= lambda x:x.strip("@"))
regions = pd.json_normalize(pl[pl.Name == "Regions"]["NODE"].tolist()[0]).rename(columns= lambda x:x.strip("@"))
energies = pd.json_normalize(pl[pl.Name == "Energies"]["NODE"].tolist()[0]).rename(columns= lambda x:x.strip("@"))
vibes = pd.json_normalize(pl[pl.Name == "Vibes"]["NODE"].tolist()[0]).rename(columns= lambda x:x.strip("@"))

Clean up the `TRACK` column by extracting just the rekordbox track ids.

In [125]:
genres["TrackID"] = genres.TRACK.apply(lambda x: [i["@Key"] for i in x])
genres["TrackID"].head()

0                    [268245278, 268246423, 174798994]
1    [268245352, 268245868, 268246121, 268247765, 2...
2    [268246498, 163757037, 268246506, 268246505, 2...
3    [268246483, 268246479, 268246480, 202953415, 2...
4    [268246499, 268246481, 268245279, 268246897, 2...
Name: TrackID, dtype: object

My goal here is to create a dataframe with two columns and rows represent a track id of a song that is a member of that genre playlist.

Select `GenreName` and `TrackID` columns and set `GenreName` to index.

In [126]:
genres = genres.set_index("GenreName")["TrackID"]
genres.head(2)

GenreName
Reggae                    [268245278, 268246423, 174798994]
Jazz      [268245352, 268245868, 268246121, 268247765, 2...
Name: TrackID, dtype: object

`explode` allows us to achieve this goal.

In [127]:
genres = genres.explode().reset_index()
genres.head()

,GenreName,TrackID
0,Reggae,268245278
1,Reggae,268246423
2,Reggae,174798994
3,Jazz,268245352
4,Jazz,268245868


Let's merge `genres` with the metadata in `tags_rbox_ij`

In [131]:
tags_and_genres = pd.merge(genres, tags_rbox_ij, on ='TrackID')
tags_and_genres.head()

,GenreName,TrackID,sid,artist,length,title,bpm,initialkey,file_path,file_path_unicode,...,Genre,Size,TotalTime,AverageBpm,PlayCount,Location,TEMPO,Tonality,Location_fixed,Location_unicode
0,Reggae,268245278,329f6da0c9fac17c90922e0db818eff1,Lord Echo,288.031921,Bohemian Idol (DJ Day's Chair-Bro Remix),90.0,1A,/Volumes/LaCie/Collection/Lord Echo - Bohemian...,/Volumes/LaCie/Collection/Lord Echo - Bohemian...,...,Dub,11621046,288,90.00,2,file://localhost/Volumes/LaCie/Collection/Lord...,NaN,1A,/Volumes/LaCie/Collection/Lord Echo - Bohemian...,/Volumes/LaCie/Collection/Lord Echo - Bohemian...
1,Reggae,268246423,1275803e809e655d2771131b34e5305b,Capital Letters,350.389130,Smoking My Ganja (Peter Croce Rework),123.0,11A,/Volumes/LaCie/Collection/Capital Letters - RS...,/Volumes/LaCie/Collection/Capital Letters - RS...,...,,61824532,350,123.00,4,file://localhost/Volumes/LaCie/Collection/Capi...,NaN,12A,/Volumes/LaCie/Collection/Capital Letters - RS...,/Volumes/LaCie/Collection/Capital Letters - RS...
2,Boogie,268246423,1275803e809e655d2771131b34e5305b,Capital Letters,350.389130,Smoking My Ganja (Peter Croce Rework),123.0,11A,/Volumes/LaCie/Collection/Capital Letters - RS...,/Volumes/LaCie/Collection/Capital Letters - RS...,...,,61824532,350,123.00,4,file://localhost/Volumes/LaCie/Collection/Capi...,NaN,12A,/Volumes/LaCie/Collection/Capital Letters - RS...,/Volumes/LaCie/Collection/Capital Letters - RS...
3,Reggae,174798994,91728be38c6c97b87bfd7a4e73260e50,Michael Paul,292.000000,Reggae Music,102.5,8A,/Volumes/LaCie/Collection/Michael Paul - Regga...,/Volumes/LaCie/Collection/Michael Paul - Regga...,...,,77263244,291,102.50,0,file://localhost/Volumes/LaCie/Collection/Mich...,"[{'@Inizio': '0.107', '@Bpm': '102.50', '@Metr...",8A,/Volumes/LaCie/Collection/Michael Paul - Regga...,/Volumes/LaCie/Collection/Michael Paul - Regga...
4,Jazz,268245352,ef20c3f26b08ac309f04ee0dd4ef8a2d,Fire_Flight,271.487701,Wantin'_U_PN,121.0,5A,/Volumes/LaCie/Collection/4-Fire_Flight_-_Want...,/Volumes/LaCie/Collection/4-Fire_Flight_-_Want...,...,,47901316,271,121.15,0,file://localhost/Volumes/LaCie/Collection/4-Fi...,"[{'@Inizio': '0.375', '@Bpm': '121.15', '@Metr...",5A,/Volumes/LaCie/Collection/4-Fire_Flight_-_Want...,/Volumes/LaCie/Collection/4-Fire_Flight_-_Want...


I've successfully managed to connect the rekordbox playlist data with my database data.

In [84]:
genres["TRACK_ID"] = genres.TRACK.apply(lambda x: [i["@Key"] for i in x])
regions["TRACK_ID"] = regions.TRACK.apply(lambda x: [i["@Key"] for i in x])
energies["TRACK_ID"] = energies.TRACK.apply(lambda x: [i["@Key"] for i in x])
vibes["TRACK_ID"] = vibes.TRACK.apply(lambda x: [i["@Key"] for i in x])

In [85]:
genres = genres[["Name", "TRACK_ID"]].set_index("Name")
regions = regions[["Name", "TRACK_ID"]].set_index("Name")
energies = energies[["Name", "TRACK_ID"]].set_index("Name")
vibes = vibes[["Name", "TRACK_ID"]].set_index("Name")

In [86]:
genres = genres.TRACK_ID.explode().reset_index()
regions = regions.TRACK_ID.explode().reset_index()
energies = energies.TRACK_ID.explode().reset_index()
vibes = vibes.TRACK_ID.explode().reset_index()

In [87]:
genres = pd.merge(genres, joined_inner[["TrackID", "sid"]] ,left_on="TRACK_ID", right_on="TrackID")
regions = pd.merge(regions, joined_inner[["TrackID", "sid"]] ,left_on="TRACK_ID", right_on="TrackID")
energies = pd.merge(energies, joined_inner[["TrackID", "sid"]] ,left_on="TRACK_ID", right_on="TrackID")
vibes = pd.merge(vibes, joined_inner[["TrackID", "sid"]] ,left_on="TRACK_ID", right_on="TrackID")